In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.applications.vgg16 import VGG16
from keras import models, layers, optimizers
from keras.callbacks import ModelCheckpoint
from PIL import Image, ImageChops, ImageEnhance
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder

def convert_to_ela_image(path, quality):
    filename = path
    resaved_filename = filename.split('.')[0] + '.resaved.jpg'
    im = Image.open(filename).convert('RGB')
    im.save(resaved_filename, 'JPEG', quality=quality)
    resaved_im = Image.open(resaved_filename)
    ela_im = ImageChops.difference(im, resaved_im)
    extrema = ela_im.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff

    ela_im = ImageEnhance.Brightness(ela_im).enhance(scale)
    return ela_im

def read_and_process_data(dataset_path):
    X = []
    Y = []
    for class_folder in ['Au', 'Tp']:
        class_path = os.path.join(dataset_path, class_folder)

        if not os.path.exists(class_path):
            print(f"Folder '{class_folder}' not found in '{dataset_path}'. Please check your folder structure.")
            return

        for image_name in os.listdir(class_path):
            if image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                image_path = os.path.join(class_path, image_name)
                ela_image = convert_to_ela_image(image_path, 90)
                ela_array = np.array(ela_image.resize((224, 224))) / 255.0
                X.append(ela_array)
                label_encoder = LabelEncoder()
                Y.append(label_encoder.fit_transform([class_folder]))

    X = np.array(X)
    Y = to_categorical(Y, num_classes=2)
    X = X.reshape(-1, 224, 224, 3)

    X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.1, random_state=5, shuffle=True)

    return X_train, X_val, Y_train, Y_val

def create_vgg16_model(input_shape):
    vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    model = models.Sequential()
    model.add(vgg_conv)
    model.add(layers.Flatten())
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(2, activation='softmax'))
    return model

def train_vgg16_model(dataset_path, lr, ep):
    X_train, X_val, Y_train, Y_val = read_and_process_data(dataset_path)

    input_shape = (224, 224, 3)
    model = create_vgg16_model(input_shape)

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.RMSprop(lr=lr),
                  metrics=['accuracy'])

    epochs = ep
    batch_size = 20

    checkpoint = ModelCheckpoint("vgg16_best_model.h5", monitor='val_accuracy', verbose=1, save_best_only=True,
                                 mode='max')

    history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
                        validation_data=(X_val, Y_val), verbose=2, callbacks=[checkpoint])


    return history

dataset_path = '/content/drive/MyDrive/CASIA2.0_revised'
learning_rate = 0.001
num_epochs = 10
train_vgg16_model(dataset_path, learning_rate, num_epochs)
